# CS535/EE514 Machine Learning - Spring 2024 - Assignment 5

# Support Vector Machines

**Date Assigned**: Saturday, April 06, 2024

**Date Due**: Monday, April 22, 2024 (11:55 pm)

**Important Notes**

1.   The assignment integrates tasks spanning methods as well as principles. Some tasks will involve implementation (in Python) and some may require mathematical analysis.  
2.   All cells must be run once before submission and should be displaying the results(graphs/plots etc). Failure to do so will result in deduction of points.
3.   While discussions with your peers is strongly encouraged, please keep in mind that the assignment is to be attempted on an individual level. Any plagiarism (from your peers) will be referred to the DC without hestitation.
4. For tasks requiring mathematical analysis, students familiar with latex may type their solutions directly in the appropriate cells of this notebook. Alternatively, they may submit a hand-written solution as well.
5. Use procedural programming style and comment your code properly.
5. Upload your solutions as a zip folder with name `RollNumber_A5` on the Assignment tab and submit your hand-written solutions in the drop-box next to the instructor's office.
5. **Policy on Usage of Generative AI Tools**. Students are most welcome to use generative AI tools as partners in their learning journey. However, it should be kept in mind that these tools cannot be blindly trusted for the tasks in this assignment (hopefully) and therefore it is important for students to rely on their own real intelligence (pun intended) before finalizing their solution/code. It is also mandatory for students to write a statement on how exactly have they used any AI tool in completing this assignment.
5. **Vivas** The teaching staff reserves the right to conduct a viva for any student.   
5. **Policy on Late Submission**. Late solutions will be accepted with a 15% penalty per day till Wednesday, April 24, 2024 (11:55 pm) . No submissions will be accepted after that.      




## Task 1. Support Vector Machines Implementation

All the necessary imports have been given below. If you are running the assignment locally, you may comment out the pip command below to install the QP solver for SVMs. If you need to import anything else, feel free to reach out.

In [138]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
import matplotlib as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.svm import SVC
!pip install cvxopt
import cvxopt # The optimization package for Quadratic Programming
import cvxopt.solvers
from sklearn.base import BaseEstimator, ClassifierMixin

import numpy as np

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


We will be using the make moon dataset for this assignment. We will be first implementing it with the help of library functions and then through our own implementation and compare the results. The following code snippet has been provided for you which divides the dataset into train and test. Please do not modify it.

In [139]:
X, y = make_moons (n_samples = 500, noise = 0.15, random_state = 42)
y = y*2-1.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Visualization function


In [140]:
def visualization_function (classifier, feature_matrix, label_vector, axes=[-2, 3, -2, 2]):
    """
    Generate a simple plot of SVM including the decision boundary, margin, and its training data

    Parameters
    ----------
    classifier: your classifier handle
    feature_matrix: feature matrix shape(m_samples, n_features)
    label_vector: label vector shape(m_samples, )
    axes: (optional) the axes of the plot in format [xmin, xmax, ymin, ymax]
    """

    # Create a mesh grid based on the provided axes (100 x 100 resolution)
    axes_x = np.linspace(axes[0], axes[1], 100)
    axes_y = np.linspace(axes[2], axes[3], 100)
    x, y = np.meshgrid(axes_x, axes_y)

    # predicting decision boundary
    x_new = np.c_(x.ravel(), y.ravel())
    y_pred = classifier.predict(x_new).reshape(x.shape)

    # Plot decision boundary and margins
    plt.figure(figsize=(10, 5))
    plt.contour(x, y, y_pred, alpha=0.2, cmap=plt.cm.brg)

    # Plot out the support vectors
    support_vectors = classifier.support_vectors_
    plt.scatter(support_vectors[:, 0], support_vectors[:, 1], linewidth=1, facecolors='none', edgecolors='k')

    # Plot out the training data
    plt.scatter(feature_matrix[:, 0], feature_matrix[:, 1], c=label_vector, cmap=plt.cm.brg, edgecolors='k')

    plt.title("SVM plot with decision boundary, margin, and its training data")
    plt.xlable('Feature 1')
    plt.ylable('Feature 2')
    plt.axis(axes)
    plt.grid(True)
    plt.show()

## SVM Classifier Implementation and Evaluation using Built in Library SVC

### Kernels to Implement
- **Linear Kernel**
- **Polynomial Kernel**
- **Gaussian Radial Basis Function (RBF) Kernel**

### Procedure

#### 1. Hyperparameter Optimization
- Utilize grid search to fine-tune the model parameters.
- Focus on the regularization parameter `C` and the kernel-specific parameter `gamma` (where applicable).

#### 2. Model Training and Evaluation
- Train the SVM classifier using the hyperparameters obtained from the optimization step.
- Evaluate the trained model on a separate test dataset to determine its performance.

#### 3. Metrics Reporting
- Compile and report the following metrics to assess the classifier's performance:
  - Confusion matrix
  - Recall
  - Precision

#### 4. Visualization
- Use the predefined `visualization_function()` function to visualize the SVM’s decision boundaries and support vectors in 2D.
  This will provide insights into how the SVM constructs the decision plane and identifies support vectors across different kernels.




In [141]:
def svm_evaluation_library_functions(X_train, y_train, X_test, y_test):
    """
    Tune and evaluate SVM with different kernels.
    1. Define the parameter grid for each kernel type.
    2. Perform grid search to find the best hyperparameters.
    3. Train the model using the best hyperparameters and evaluate it.
    4. Display the evaluation metrics and visualize the SVM.
    """

    # 1.
    parameter_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly']
    }

    # 2.
    grid_search = GridSearchCV(SVC(), param_grid=parameter_grid, scoring='accuracy', cv=5)
    grid_search.fit(X_train, y_train)

    # 3.
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # 4.
    print(f"Recall: {recall:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print("Confusion Matrix:\n", conf_matrix)

    if X_train.shape[1] == 2:
        pass
        # visualization_function(best_model, X_train, y_train)

## Custom non Linear SVM implementation

In [142]:
def linear_kernel(vector_a, vector_b):
    """
    Computes the linear kernel between two vectors.
    1. The function should take two vectors, vector_a and vector_b, as input.
    2. It calculates the dot product of these vectors.
    3. The result is returned as the linear kernel value.
    """
    # making sure they are numpy arrays
    vector_a = np.array(vector_a)
    vector_b = np.array(vector_b)

    # just the dot product between the input vectors
    linear_kernel_value = np.dot(vector_a, vector_b)

    return linear_kernel_value
    
def polynomial_kernel(vector_a, vector_b, degree=None):
    """
    Computes the polynomial kernel between two vectors.
    1. The function takes two vectors, vector_a and vector_b, and the polynomial degree as input.
    2. It calculates the polynomial kernel by raising the dot product of vector_a and vector_b to the specified power of degree.
    3. The result is returned as the polynomial kernel value, indicating the similarity in a polynomial feature space.
    """
    # making sure they are numpy arrays
    vector_a = np.array(vector_a)
    vector_b = np.array(vector_b)

    # calculating polynomial kernel
    polynomial_kernel_value = 1 + np.dot(vector_a, vector_b)
    if degree:
        polynomial_kernel_value = polynomial_kernel_value ** degree
    
    return polynomial_kernel_value

def rbf_kernel(vector_a, vector_b, gamma=None):
    """
    Computes the Gaussian RBF kernel between two vectors.
    1. The function takes two vectors, vector_a and vector_b, and the parameter gamma as input.
       Gamma is a key parameter in the RBF kernel which determines the influence of individual training samples
       on the decision boundary. Specifically, it controls the width of the Gaussian function:
       - A low gamma value makes the decision boundary smoother, with a wider spread of influence for each support vector.
       - A high gamma value creates a more complex model, where influence is limited to vectors close to the support vector,
         leading to a decision boundary that closely follows the training points.
    2. It calculates the Gaussian RBF kernel using the squared Euclidean distance between vector_a and vector_b, scaled by gamma.
    3. The result, an exponential function of the negative scaled squared distance, is returned as the RBF kernel value,
       depicting the similarity in the RBF feature space.
    """
    # making sure they are numpy arrays
    vector_a = np.array(vector_a)
    vector_b = np.array(vector_b)

    # calculating kernel
    gaussian_rbf_kernel = (np.linalg.norm(vector_a - vector_b)**2)
    if gamma:
        gaussian_rbf_kernel *= (-gamma)
    rbf_kernel_value = np.exp(gaussian_rbf_kernel)

    return rbf_kernel_value

## Implementing the SVM Class

In [143]:
class NonLinearSVM (BaseEstimator, ClassifierMixin):
    """
    Custom implementation of the Support Vector Machine (SVM) algorithm
    The class utilizes the quadratic programming solver from cvxopt to optimize the SVM formulation.
    """

    def __init__(self, kernel=None, regularization_parameter=None):
        """
        Initializes the NonLinearSVM instance with the specified kernel function and regularization parameter.

        Parameters:
        kernel: A function that computes the kernel between two data points. The kernel function defines the feature
            space in which the dataset is transformed, allowing SVM to perform classification in this space.
        regularization_parameter: The regularization parameter C controls the trade-off between achieving a low error on the
            training data and minimizing the model complexity for better generalization. If C is set to None,
            it implies an infinite margin (hard margin SVM). A larger C tries to classify all training examples
            correctly (hard margin), while a smaller C allows more misclassifications but might lead to a better generalization (soft margin).
        """
        if kernel == "rbf":
            self.kernel = rbf_kernel
        elif kernel == "poly":
            self.kernel = polynomial_kernel
        elif kernel == "linear":
            self.kernel = linear_kernel

        self.C = regularization_parameter

    def fit(self, feature_matrix, target_vector):
        """
        Trains the SVM model using the provided training data.
        Parameters:
        feature_matrix : array-like, shape (total_num_samples, total_num_features)
            The training input samples.
        target_vector : array-like, shape (total_num_samples,)
            The target values (class labels) relative to the input samples.

        The training process involves several steps:
        a. It computes the kernel matrix, where each element is the result of applying the kernel function to a pair of samples.
        b. It sets up and solves a quadratic programming problem to find the Lagrange multipliers.
        c. It identifies the support vectors as those samples corresponding to non-zero Lagrange multipliers.
        d. In the case of a linear kernel, it computes the weight vector w. For non-linear kernels, w is not used.
        e. It computes the intercept term b using the support vectors and their properties.

        Starter code has been provided for this. Fill in the dotted lines and code below each comment provided.
        """

        # Compute the number of samples and features
        total_num_samples, total_num_features = feature_matrix.shape

        # a. Compute the kernel matrix for all pairs of training examples
        # Kernel function is applied to each pair of samples in the training set
        K = np.zeros((total_num_samples, total_num_features))
        for i in range(total_num_samples):
            for j in range(total_num_features):
                K[i, j]  =  self.kernel(feature_matrix[i], feature_matrix[j])  

        # b. Setup the Quadratic Programming (QP) problem for SVM
        
        # quadratic_term: Quadratic term in the objective, derived from the outer product of the target vector and kernel matrix
        # quadratic_term = cvxopt.matrix(np.outer(target_vector, target_vector) * K)  # P
        quadratic_term = cvxopt.matrix(np.outer(target_vector, target_vector))  # P
        P = quadratic_term

        # linear_term: Linear term in the objective, a vector of -1's
        linear_term = cvxopt.matrix(-np.ones((total_num_samples, 1)))   # q
        q = linear_term

        # equality_constraint_coeff: Equality constraint coefficient for the problem, converting target vector to meet cvxopt requirements
        equality_constraint_coeff = cvxopt.matrix(target_vector.reshape(1, -1)) # A
        A = equality_constraint_coeff

        # equality_constraint_threshold: Equality constraint threshold, set to 0 to enforce the constraint that sum(alpha_i * y_i) = 0
        equality_constraint_threshold = cvxopt.matrix(np.zeros(1))  # b
        b = equality_constraint_threshold

        # For the SVM problem, constraints are defined for Lagrange multipliers (alpha)
        # For hard margin SVM, constraints ensure alpha >= 0
        # For soft margin SVM (when C is defined), constraints allow 0 <= alpha <= C
        # You will define two variables for this: an inequality constraint matrix and an inequality constraint vector.
        # You will check whether the regularization parameter has been defined to see if its hard or soft margin.

        # G & h
        if self.C:
            inequality_constraint_matrix = cvxopt.matrix(np.vstack((-np.eye(total_num_samples), np.eye(total_num_samples))))
            inequality_constraint_vector = cvxopt.matrix(np.hstack((np.zeros(total_num_samples), np.ones(total_num_samples) * self.C)))
        else:
            inequality_constraint_matrix = cvxopt.matrix(-np.eye(total_num_samples))
            inequality_constraint_vector = cvxopt.matrix(np.zeros(total_num_samples))

        G = inequality_constraint_matrix
        h = inequality_constraint_vector

        # Solve the quadratic programming problem using cvxopt
        cvxopt.solvers.options['show_progress'] = False
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Extract the Lagrange multipliers from the solution
        alphas = np.ravel(solution['x'])

        # Support vectors have non-zero Lagrange multipliers
        support_vectors = (alphas > 0).flatten()

        # Identify these support vectors for later use in the model
        indices = np.arange(len(alphas))[support_vectors]
        self.alphas = alphas[support_vectors]
        self.sv = feature_matrix[support_vectors]
        self.sv_y = target_vector[support_vectors]

        # Code part d and e here

        # d & E 
        if self.kernel == 'linear':
            self.w = np.dot((target_vector * alphas).T @ feature_matrix).reshape(-1, 1)
            self.b = np.mean(self.sv_y - np.dot(self.sv, self.w))
        else:
            self.w = None
            self.b = np.mean(self.sv_y - np.sum(self.sv * self.alphas * K[indices][:, self.sv], axis=1))

def decision_function(self, feature_matrix):
    """
    Calculates the signed distance of samples in feature_matrix from the decision boundary defined by the SVM model.

    Parameters:
    feature_matrix : array-like, shape (total_num_samples, total_num_features)
        Feature matrix for which to predict the distances from the decision boundary.

    Returns:
    distances : array, shape (total_num_samples,)
        The signed distances of each sample in feature_matrix from the decision boundary.

    The decision function is defined differently based on the kernel used:
    - For a linear kernel, it's the dot product of the input features and the weight vector plus the intercept.
    - For non-linear kernels, the decision function involves applying the kernel function between the input features
      and the support vectors, weighted by the learned Lagrange multipliers and the labels of the support vectors.

    Specifically:
    - If the linear kernel is used (self.w is not None), the decision function calculates as w^T * feature_matrix + b.
    - For non-linear kernels, the function sums the product of the Lagrange multipliers (self.a),
      the labels of the support vectors (self.y), and the kernel function applied between each support vector
      and the input features. The intercept (self.b) is added to this sum to obtain the final decision function value.
    """
    if self.kernel == linear_kernel:
        return np.dot(self.w.T, feature_matrix) + self.b
    else:
        sum_ = np.sum(self.a * self.y * self.kernel)
        return  sum_ + self.b

def predict(self, X):
    """
    The predict method classifies samples in X based on the sign of the decision function, outputting -1 or +1.
    """
    decision = self.decision_function(X)

    class_ = np.sign(decision).astype(int)
    return class_


## Using the nonLinearSVM class to train the models and generating a classification report

In [144]:
def train_evaluate_plot_svm(X_train, y_train, X_test, y_test):
    """
    Trains and evaluates the custom SVM with different kernels.
    1. Create MySVM instances with linear, polynomial, and RBF kernels.
    2. Use the best hyperparameters found through the library implementation.
    3. Train the SVM on the training data using these hyperparameters.
    4. Evaluate the trained SVM on the test data and print the precision, accuracy, and recall.
    5. Use the visualization function to plot the decision boundaries.
    """
    kernels = ['linear', 'polynomial', 'rbf']
    params = {
        'C': [1, 10, 100],
        'kernel': kernels,
        'degree': [2, 3],
        'gamma': ['scale', 'auto']
    }

    grid_search = GridSearchCV(SVC(), params, cv=3, scoring='accuracyr')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"Recall: {recall:.2f}")
    print(f"Precision: {precision:.2f}")
    print("Confusion Matrix:\n", conf_matrix)

    if X_train.shape[1] == 2:
        visualization_function(best_model, X_train, y_train)

In [145]:
# initializing the kernel
model_ = SVC(kernel='rbf', C=1.0, gamma='scale') 

# training the model
model_.fit(X_train, y_train)
y_pred = model_.predict(X_test)

# calculating accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Model accuracy on test data: {accuracy}')

Model accuracy on test data: 0.98


In [146]:
# initializing the kernel
model__ = NonLinearSVM(kernel='rbf', regularization_parameter=1.0) 

# training the model
model_.fit(X_train, y_train)
y_pred = model_.predict(X_test)

# calculating accuracy
svm_evaluation_library_functions(X_train, y_train, X_test, y_test)


Recall: 1.00
Precision: 1.00
Accuracy: 1.00
Confusion Matrix:
 [[43  0]
 [ 0 57]]


## Comparison Questions

1. How does the accuracy of your custom SVM implementation compare with scikit-learn's SVC?
    
    Comparing the accuracy of a custom SVM implementation with scikit-learn's SVC typically reveals that scikit-learn's implementation achieves higher accuracy in most cases. This difference in accuracy can arise due to various factors such as optimization techniques, handling of numerical stability, and parameter tuning.
2. How sensitive is each model to hyperparameter changes, particularly C and gamma? Perform a detailed analysis by varying these parameters and observing the impact on performance metrics like accuracy, precision, and recall. 

Both the custom SVM implementation and scikit-learn's SVC are sensitive to hyperparameter changes, particularly the regularization parameter C and the kernel parameter gamma. Varying these parameters can have a significant impact on model performance metrics such as accuracy, precision, and recall. Typically, increasing C n both implementations leads to a tighter decision boundary and potentially overfitting, while increasing gamma an make the decision boundary more complex and prone to overfitting in the case of kernel SVMs.

3. Investigate how the selection of kernel parameters (like degree in the polynomial kernel) affects the decision boundary in both the custom and scikit-learn SVMs. Can you visualize the differences in decision boundaries for various parameter settings?

The selection of kernel parameters, such as the degree in the polynomial kernel, significantly affects the decision boundary in both custom and scikit-learn SVMs. Higher degrees in polynomial kernels can result in more complex decision boundaries, which may lead to overfitting if not properly tuned. Visualizing the decision boundaries for various parameter settings can provide insights into how the models generalize and separate classes in the feature space.

## Task 2. Multi-Class SVM

In order to extend the binary SVM classifier discussed in class to $C$ classes, consider the following one-versus rest proposal.

Train a binary SVM (possibly non-linear) classifier by first treating class-$0$  points on one side and the remaining on the other. Let the hypothesis function be denoted by $h_0(.)$. Thus, for a test point $\vec{x}_t$, the function $h_0(\vec{x}_t)$ returns a 1 if it thinks the point is class-0 and returns a 0 otherwise.

Repeat the procedure for every class to ultimately obtain $C$ hypothesis functions $h_0(.), h_1(.), \ldots, h_{C-1}(.)$.

Armed with these hypothesis functions, Khurram claims that the multi-class SVM formulation is complete: To determine whether a test point belongs to any class-$i$ or not, he claims he will simply evaluate $\vec{x}_t$, he claims he will evaluate $h_i(\vec{x}_t)$ and if it returns 1, we know the point is class-$i$ - Mission accomplished.

1. Explain why Khurram's classifier will not always work.
2. Devise a solution to overcome the problem you identify with Khurram's method.

![image.png](task2.png)

## Task 3. Kernelized Perceptron

Recall that we could induce non-linear boundaries with SVM by projecting the features $\vec{x}$ to the vector $\phi(\vec{x})$ that resides in a higher dimensional space. To reduce the computational complexity associated with computing inner products in the high dimensional space, we resort to using kernel functions $k(\vec{x}_i, \vec{x}_j) = \phi(\vec{x}_i)^T \phi(\vec{x}_j)$.

The same idea could be used to induce a non-linear classifer with Perceptron as well: Map all $\vec{x}_i$ to the vector $\phi(\vec{x}_i)$ and then determine a linear decision boundary in the higher dimensions - the linear boundary in the higher dimensions would induce a non-linear one in the original feature space. To reduce the computational complexity, we would however like to resort to using kernel functions for Perceptron as well. This task will have you derive a kernelized version of the Perceptron algorithm.

1. With the initialization $\vec{w} = 0$, show that at any iteration, $\vec{w} = \sum_{i=j}^N \alpha_j \phi(\vec{x}_j)$.
2. Show that as a consequence of point 1 above, we have

$$\vec{w}^T \vec{x}_i = \sum_{j=1}^N \alpha_j k(\vec{x}_j, \vec{x}_i) $$

Thus, without even knowing the transformation, the inner product can be computed if we just keep track of and update the weights $\alpha_1, \ldots, \alpha_N$.

3. Given the information above, write a Pseudo-code (similar to the one in class notes) for a kernelized version of Perceptron.



![image.png](task3.png)

## Task 4.

Consider the following hard-SVM problem formulation. A SVM classifier aiming to maximize the margin is defined by the optimization problem:

$$
\min_{\vec{w}, b} \|\vec{w}\|^2 \quad
$$

subject to the constraints:

$$
y_i(\vec{w}^T \vec{x}_i + b) \geq 1, \quad \forall \quad i=1,\ldots,N \quad
$$


Suppose that the constraints are now modified to

$$
y_i(\vec{w}^T \vec{x}_i + b) \geq c, \quad \forall \quad i=1,\ldots,N \quad
$$

for any positive constant $c$. Show that the maximum margin hyperplane obtained with this modification will exactly be the same as the one obtained without the modification. In other words, the optimization result is independent of $c$.

*Hint*. Refer to the steps followed in class in deriving the optimization problem above.

![alt text](task4.png)

## Task 5.

Consider the following soft-SVM problem formulation. The SVM classifier is defined by the optimization problem:

$$
\min_{\vec{w}, b} \|\vec{w}\|^2 + C \sum_{i=1}^N \zeta_i
$$

subject to the constraints:

$$
y_i(\vec{w}^T \vec{x}_i + b) \geq 1 - \zeta_i, \quad \forall \quad i=1,\ldots,N \quad
$$

$$ \zeta_i \geq 0 \quad \forall \quad i=1,\ldots,N \quad$$

Where $C \geq 0$ is a hyperparameter that controls the tradeoff between the penalty and the margin term. Why does $C$ have to be non-negative? Answer this by stating how the optimization problem will behave when $C < 0$.

![Image Alt Text](task5.jpg)
